In [1]:
import sys
import os

# Add the utils directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'utils')))

import pandas as pd
import modin.pandas as mpd
from utils import clickhouse_data
import numpy as np
import talib
from sklearn.decomposition import PCA

# Define parameters
start_date = '2017-01-01'
end_date = '2025-12-31'

# Load and preprocess data
large_cap = clickhouse_data.clickhouse_largecap(start_date, end_date)
# mid_cap = clickhouse_data.clickhouse_midcap(start_date, end_date)
# small_cap = clickhouse_data.clickhouse_smallcap(start_date, end_date)

# # Fill missing dates
# large_cap = clickhouse_data.fill_missing_dates_modin_optimized(large_cap)
# mid_cap = clickhouse_data.fill_missing_dates_modin_optimized(mid_cap)
# small_cap = clickhouse_data.fill_missing_dates_modin_optimized(small_cap)

# Stack dataframes
all_cap = mpd.concat([large_cap])



    SELECT 
        financialinstrumentid,
        date,
        open,
        high,
        low,
        close,
        lastprice,
        previouscloseprice,
        volume,
        totaltradingvolume,
        totaltradevalue,
        totalnumberoftradesexecuted,
        tickersymbol,
        securityseries,
        settlementprice,
        financialinstrumentname
    FROM stock_data.tickers
    WHERE financialinstrumentid IN ('500002',) AND date BETWEEN '2017-01-01' AND '2025-12-31'
    ORDER BY financialinstrumentid, date ASC
    


In addition, using fork() with Python in general is a recipe for mysterious
deadlocks and crashes.

The most likely reason you are seeing this error is because you are using the
multiprocessing module on Linux, which uses fork() by default. This will be
fixed in Python 3.14. Until then, you want to use the "spawn" context instead.

See https://docs.pola.rs/user-guide/misc/multiprocessing/ for details.

If you really know what your doing, you can silence this warning with the warning module
or by setting POLARS_ALLOW_FORKING_THREAD=1.

2025-02-28 01:47:19,889	INFO worker.py:1821 -- Started a local Ray instance.


# Feature Engineering functions with validations and importance. 


In [2]:
import numpy as np
import pandas as pd
import talib
from prophet import Prophet

def extract_all_features(df):
    """
    Master function that:
    1. Retains raw OHLCV & additional trading data.
    2. Computes trend-based, volatility, liquidity, and microstructure features.
    3. Computes advanced technical indicators.
    4. Extracts seasonal components using Prophet.
    5. Ensures all feature columns are complete and normalized.
    """
    
    df = df.copy()

    # === Retain Raw Data for Reference ===
    raw_cols = [
        'date', 'open', 'high', 'low', 'close', 'lastprice', 'previouscloseprice',
        'volume', 'totaltradingvolume', 'totaltradevalue', 'totalnumberoftradesexecuted'
    ]
    df = df[raw_cols]  # Ensure we only work with the relevant columns

    # Ensure proper datetime format
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)

    # === Compute Derived Features ===
    
    ## 🔹 Trend-Based Features
    df = clickhouse_data.compute_trend_features(df)

    ## 🔹 Volatility Features
    df = clickhouse_data.compute_volatility_features(df)

    ## 🔹 Liquidity Features (Includes Total Trading Volume & Trade Value)
    df = clickhouse_data.compute_liquidity_features(df)

    ## 🔹 Market Microstructure Features (VWAP, etc.)
    df = clickhouse_data.compute_microstructure_features(df)

    # === Derived Features from Additional Trading Data ===

    # 🔹 Previous Close Return (Gap Indicator)
    df['prev_close_return'] = np.log(df['close'] / df['previouscloseprice'])

    # 🔹 Trading Intensity (Total Trades / Volume)
    df['trading_intensity'] = df['totalnumberoftradesexecuted'] / df['totaltradingvolume']

    # 🔹 Turnover Ratio (Liquidity Proxy)
    df['turnover_ratio'] = df['totaltradevalue'] / df['totaltradingvolume']

    # 🔹 VWAP Ratio (Price Positioning)
    df['vwap_ratio'] = df['close'] / df['vwap']
    
    # 🔹 High-Low & Close-Open Ratios
    df['high_low_ratio'] = df['high'] / df['low']
    df['close_open_ratio'] = df['close'] / df['open']

    # === Advanced Technical Indicators ===
    df['cci_20'] = talib.CCI(df['high'].values, df['low'].values, df['close'].values, timeperiod=20)
    df['williams_r'] = talib.WILLR(df['high'].values, df['low'].values, df['close'].values, timeperiod=14)
    df['stoch_k'], df['stoch_d'] = talib.STOCH(
        df['high'].values, df['low'].values, df['close'].values, 
        fastk_period=14, slowk_period=3, slowd_period=3
    )
    df['chande_momentum'] = talib.CMO(df['close'].values, timeperiod=14)
    df['ulcer_index'] = np.sqrt((df['close'].rolling(14).max().values - df['close'].values) ** 2 / 14)
    df['obv'] = talib.OBV(df['close'].astype(float).values, df['volume'].astype(float).values)
    df['chaikin_money_flow'] = talib.ADOSC(
        df['high'].astype(float).values, df['low'].astype(float).values, df['close'].astype(float).values, df['volume'].astype(float).values, 
        fastperiod=3, slowperiod=10
    )
    
    # === Extract Seasonality Features Using Prophet ===
    df_prophet = df[['close']].reset_index()
    df_prophet.columns = ['ds', 'y']

    prophet = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=False)
    prophet.fit(df_prophet)

    future = prophet.make_future_dataframe(periods=0)
    forecast = prophet.predict(future)

    df['seasonal_weekly'] = forecast['weekly']
    df['seasonal_yearly'] = forecast['yearly']
    
    # Fill missing values (if any)
    df = df.ffill()
    df.fillna(0, inplace=True)

    return df

featured_df = extract_all_features(all_cap)



Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.
01:47:28 - cmdstanpy - INFO - Chain [1] start processing
01:47:29 - cmdstanpy - INFO - Chain [1] done processing


In [3]:

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

# 🚀 Enable Mixed Precision for Faster Training on M2 GPU
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# 🚀 Input Data Validation: Ensure there are no NaNs
if featured_df.isnull().values.any():
    raise ValueError("🚨 Warning: Input Data Contains NaN Values! Please handle missing values.")

# 🚀 Load and scale the data
X = featured_df.dropna().astype(np.float32)  # Ensure correct dtype
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
input_dim = X_scaled.shape[1]

# ✅ Define a Simple Autoencoder Model
encoding_dim = min(10, input_dim // 2)  # Default to 10 or half of input features, whichever is smaller

# ✅ Encoder
input_layer = Input(shape=(input_dim,))
x = Dense(encoding_dim * 4, activation='relu', kernel_initializer='he_normal')(input_layer)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Dense(encoding_dim * 2, activation='relu', kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
encoded = Dense(encoding_dim, activation='relu', kernel_initializer='he_normal', name="bottleneck")(x)  # Latent space

# ✅ Decoder
x = Dense(encoding_dim * 2, activation='relu', kernel_initializer='he_normal')(encoded)
x = BatchNormalization()(x)
x = Dense(encoding_dim * 4, activation='relu', kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
decoded = Dense(input_dim, activation='linear')(x)  # Reconstruct input

# ✅ Build Model
autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss=tf.keras.losses.Huber(delta=1.0))

# ✅ Manually Split the Data
split_ratio = 0.9
split_index = int(X_scaled.shape[0] * split_ratio)
X_train, X_val = X_scaled[:split_index], X_scaled[split_index:]

# ✅ Train Autoencoder Using NumPy Arrays
autoencoder.fit(X_train, X_train, 
                validation_data=(X_val, X_val),  # ✅ Explicitly define validation data
                epochs=50, 
                batch_size=32)

# ✅ Extract Latent Features (Encoder Model)
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer("bottleneck").output)
X_encoded = encoder.predict(X_scaled)

# ✅ Convert Encoded Features to DataFrame
encoded_features_df = pd.DataFrame(X_encoded, index=featured_df.index)

print("✅ Simple Feature Extraction Complete!")
encoded_features_df.head()


2025-02-28 01:47:40.401679: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-02-28 01:47:40.401730: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-02-28 01:47:40.401757: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-02-28 01:47:40.401791: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-02-28 01:47:40.401820: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/50


2025-02-28 01:47:42.132611: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-02-28 01:47:42.144407: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:966] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.


52/52 ━━━━━━━━━━━━━━━━━━━━ 8s 63ms/step - loss: 0.4451 - val_loss: 0.9809
Epoch 2/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 0.2385 - val_loss: 0.6727
Epoch 3/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - loss: 0.1691 - val_loss: 0.5432
Epoch 4/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 0.1509 - val_loss: 0.4608
Epoch 5/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 0.1367 - val_loss: 0.4411
Epoch 6/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 86ms/step - loss: 0.1282 - val_loss: 0.4563
Epoch 7/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 76ms/step - loss: 0.1255 - val_loss: 0.4339
Epoch 8/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - loss: 0.1177 - val_loss: 0.4600
Epoch 9/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - loss: 0.1163 - val_loss: 0.4107
Epoch 10/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 69ms/step - loss: 0.1104 - val_loss: 0.4352
Epoch 11/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - loss: 0.1081 - val_loss: 0.4355
Epoch 12/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - loss: 0.1036 - val_l

2025-02-28 01:50:00.208450: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:966] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.


58/58 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step
✅ Simple Feature Extraction Complete!


,0,1,2,3,4,5,6,7,8,9
date,,,,,,,,,,
2017-06-23,1.759766,0.806641,0.277344,0.0,7.730469,0.755859,0.470703,4.488281,0.0,0.0
2017-06-27,2.656250,0.354004,0.000000,0.0,7.953125,0.325195,0.309082,4.468750,0.0,0.0
2017-06-28,2.863281,0.329102,0.000000,0.0,7.906250,0.377930,0.366455,4.316406,0.0,0.0
2017-06-29,2.242188,0.509766,0.000000,0.0,7.933594,0.532227,0.569336,4.484375,0.0,0.0
2017-06-30,3.109375,0.293945,0.000000,0.0,7.976562,0.401367,0.493652,4.199219,0.0,0.0


In [4]:
featured_df


,open,high,low,close,lastprice,previouscloseprice,volume,totaltradingvolume,totaltradevalue,totalnumberoftradesexecuted,...,cci_20,williams_r,stoch_k,stoch_d,chande_momentum,ulcer_index,obv,chaikin_money_flow,seasonal_weekly,seasonal_yearly
date,,,,,,,,,,,,,,,,,,,,,
2017-06-23,1500.35,1503.00,1430.20,1440.55,1440.55,1486.30,10963,10963,15950701,1256,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10963.0,0.000000,0.0,0.0
2017-06-27,1445.00,1457.00,1428.85,1442.95,1442.95,1440.55,9015,9015,13001118,1135,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19978.0,0.000000,0.0,0.0
2017-06-28,1443.00,1458.00,1425.80,1452.40,1452.40,1442.95,5662,5662,8171340,892,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,25640.0,0.000000,0.0,0.0
2017-06-29,1458.00,1479.00,1435.00,1439.80,1439.80,1452.40,8416,8416,12287156,804,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,17224.0,0.000000,0.0,0.0
2017-06-30,1419.25,1460.00,1411.25,1451.10,1451.10,1439.80,4764,4764,6900488,529,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,21988.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-10,7585.00,7710.00,7572.25,7694.10,7684.50,7564.55,5208,5208,39940871,1771,...,129.803746,-2.316602,92.911017,92.425667,18.316990,0.000000,-9664466.0,6180.854158,0.0,0.0
2024-12-12,7799.95,7799.95,7635.05,7654.95,7660.15,7725.80,3840,3840,29509077,1195,...,126.512055,-12.146597,90.529267,91.351614,15.650372,10.463278,-9668306.0,4842.355629,0.0,0.0
2024-12-16,7726.30,7945.00,7709.00,7893.40,7888.00,7697.45,10909,10909,85970063,3429,...,136.401314,-3.854198,93.894201,92.444829,26.513863,0.000000,-9657397.0,5807.772359,0.0,0.0
